# Setups

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pprint import pprint
from IPython.display import display, Markdown, Pretty

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Imports

In [ ]:
from pathlib import Path

from openssa import (Agent,
                     HTP, AutoHTPlanner,
                     BaseReasoner, OodaReasoner,
                     FileResource)

# Problems & Resources

In [ ]:
PROBLEM = 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?'

In [ ]:
RESOURCE_PATH = Path() / 'FinanceBench' / '.data' / 'docs' / 'AMD_2022_10K'
assert RESOURCE_PATH.is_dir()

resource = FileResource(RESOURCE_PATH)
display(Markdown(resource.overview))

# Base Reasoning

In [ ]:
base_reasoning_agent = Agent(planner=None,
                             reasoner=BaseReasoner(),
                             resources={resource})
pprint(base_reasoning_agent)

In [ ]:
display(Markdown(base_reasoning_agent.solve(PROBLEM)))

# OODA Reasoning

In [ ]:
ooda_reasoning_agent = Agent(planner=None,
                             reasoner=OodaReasoner(),
                             resources={resource})
pprint(ooda_reasoning_agent)

In [ ]:
display(Markdown(ooda_reasoning_agent.solve(PROBLEM)))

# Agent with Planning & Reasoning

In [ ]:
agent = Agent(planner=AutoHTPlanner(max_depth=2, max_subtasks_per_decomp=3),
              reasoner=OodaReasoner(),
              resources={resource})
pprint(agent)

## Problem-Solving with Automated Planner

In [ ]:
auto_plan = agent.planner.plan(PROBLEM)
pprint(auto_plan)

In [ ]:
solution_1 = agent.solve(PROBLEM)
display(Markdown(solution_1))

## Problem-Solving with Expert-Specified Plan

In [ ]:
expert_plan = HTP.from_dict(
    {
        'task': PROBLEM,
        'sub-plans': [
            {
                'task': 'retrieve data points needed for Quick Ratio',
                'sub-plans': [
                    {
                        'task': 'retrieve Cash & Cash Equivalents'
                    },
                    {
                        'task': 'retrieve Accounts Receivable'
                    },
                    {
                        'task': 'retrieve Short-Term Liabilities'
                    },
                    {
                        'task': 'retrieve Accounts Payable'
                    },
                ]
            },
            {
                'task': 'calculate Quick Ratio'
            },
            {
                'task': 'see whether Quick Ratio is healthy, i.e. greater than 1'
            },
        ]
    }
)
pprint(expert_plan)

In [ ]:
solution_2 = agent.solve(PROBLEM, plan=expert_plan, dynamic=False)
display(Markdown(solution_2))